# q1

In [3]:
'''
there may be an error in the fourth digit in the following ISBN: 3-965-59422-9. the value of the correct digit is...
'''
def calculate_isbn_check_digit(isbn):
    total = 0
    for i, digit in enumerate(isbn[:9], start=1):
        total += i * int(digit)
    return total % 11

def find_correct_digit(isbn, error_index):
    isbn = isbn.replace("-", "")
    given_check_digit = int(isbn[-1])
    error_index -= 1 # Convert 1-based index to 0-base
    
    for replacement_digit in range(10): # Try each possible digit for the specified position
        candidate_isbn = isbn[:error_index] + str(replacement_digit) + isbn[error_index + 1:-1] # Replace the specified digit with the current guess
        calculated_check_digit = calculate_isbn_check_digit(candidate_isbn) # Calculate the check digit for the candidate ISBN
        if calculated_check_digit == given_check_digit: # Check if the calculated check digit matches the given check digit
            return replacement_digit
    raise ValueError("Could not find a valid replacement digit.")

#v v v v v v v v v v v v v v v CHANGE THIS v v v v v v v v v v v v v v v v v v v v v
isbn = "3-965-59422-9"
error_index = 4  # 1-based index of incorrect digit

print(f"correct {error_index}-th digit: {find_correct_digit(isbn, error_index)}")

correct 4-th digit: 4


# q3

In [36]:
'''
let C be the binary OR ternary linear code with parity check matrix H=...
what is the minimum distance d(C) OR min weight w(C) of the code?
OR what is the maximum number of errors that can always be corrected OR detected by C?
'''
import numpy as np
from itertools import combinations, product

def verify_minimum_distance_general(H, radix=3):
    num_columns = H.shape[1]
    
    def check_linear_combination(columns, coeffs):
        selected_cols = H[:, list(columns)]
        result = np.zeros(H.shape[0], dtype=int)
        for col, coeff in zip(selected_cols.T, coeffs):
            result = (result + coeff * col) % radix
        return np.all(result == 0)

    for r in range(2, num_columns + 1):
        for cols in combinations(range(num_columns), r):
            for coeffs in product(range(radix), repeat=r):
                if all(c == 0 for c in coeffs):  # Skip if all coefficients are 0
                    continue
                if check_linear_combination(cols, coeffs):
                    return r
                else:
                    pass
    return num_columns

#v v v v v v v v v v v v v v v CHANGE THIS v v v v v v v v v v v v v v v v v v v v v
H = np.array([
    [1,0,0,1,0,1,1],
    [0,1,1,1,1,1,0],
    [0,0,1,0,1,0,0],
    [0,0,0,1,0,0,0]
])
radix = 2

min_dist = verify_minimum_distance_general(H, radix)
print(f"d(C) or w(C): {min_dist}")
print(f'max num of errors DETECTed: {min_dist - 1}')
print(f'max num of errors CORRECTed: {(min_dist - 1) // 2}')

d(C) or w(C): 2
max num of errors DETECTed: 1
max num of errors CORRECTed: 0


# q4

In [79]:
import numpy as np

def solve_linear_system_mod2(A, b):
    A = A.copy() % 2
    b = b.copy() % 2
    nrows, ncols = A.shape
    Ab = np.hstack((A, b.reshape(-1,1))) % 2  # Augmented matrix
    rank = 0

    for col in range(ncols):
        pivot_row = None # Find a pivot in current column
        for row in range(rank, nrows):
            if Ab[row, col] == 1:
                pivot_row = row
                break
        if pivot_row is None:
            continue  # No pivot in this column
        if pivot_row != rank:  # Swap rows if necessary
            Ab[[rank, pivot_row]] = Ab[[pivot_row, rank]]
        for row in range(nrows):  # Eliminate other rows
            if row != rank and Ab[row, col] == 1:
                Ab[row] = (Ab[row] + Ab[rank]) % 2
        rank += 1

    for row in range(rank, nrows):  # Check for inconsistency
        if Ab[row, -1] == 1 and np.all(Ab[row, :-1] == 0):
            raise ValueError("No solution exists for the system.")

    x = np.zeros(ncols, dtype=int) # Back substitution
    for row in reversed(range(rank)):
        pivot_cols = np.where(Ab[row, :-1] == 1)[0]
        if len(pivot_cols) == 0:
            continue
        col = pivot_cols[0]
        x[col] = Ab[row, -1]
        if len(pivot_cols) > 1:
            x[col] = (x[col] - np.dot(Ab[row, pivot_cols[1:]], x[pivot_cols[1:]])) % 2
    return x

def encode_message(H, message_positions, check_positions, m):
    x = np.zeros(H.shape[1], dtype=int)
    x[message_positions] = m

    H_unknowns = H[:, check_positions]
    H_knowns = H[:, message_positions]
    x_knowns = x[message_positions]
    rhs = (-H_knowns.dot(x_knowns)) % 2

    x_unknowns = solve_linear_system_mod2(H_unknowns, rhs) # Solve for the check bits
    x[check_positions] = x_unknowns % 2
    return x


#v v v v v v v v v v v v v v v CHANGE THIS v v v v v v v v v v v v v v v v v v v v v
H = np.array([
    [1,1,1,0,1,0],
    [0, 0,1,1,0,0],
    [0, 0, 0,1,1, 1]
], dtype=int)
info_positions = [2,5,6]  # CAREFULLY identify info bits
check_positions = [1, 3,4] 
message = np.array([1, 1,0], dtype=int)


message_pos_to_use = [x - 1 for x in info_positions]
check_pos_to_use = [x - 1 for x in check_positions]
x1 = encode_message(H, message_pos_to_use, check_pos_to_use, message)
print("codeword:", ''.join(map(str, x1)))

codeword: 111110


# q5

In [58]:
'''
let C be ternary linear code with check matrix H =..
suppose that the codeword x was transmitted and received as the word y =..
assuming that there occured at most 1 error, correct y to find the codeword x
'''
import numpy as np

def correct_error(H, y, radix):
    y = np.array(y)  # Convert y to a numpy array for easier handling
    S_y = (H @ y.T) % radix  # Syndrome computation
    #print(f"S(y) = {S_y}")
    if np.all(S_y == 0):
        print("No error detected.")
        return y.tolist()
    for i in range(H.shape[1]):
        for scalar in range(radix): # Check if H[:,i] is equal to S_y (mod radix) or a scalar multiple of it
            if np.array_equal((H[:, i] * scalar) % radix, S_y):
                y_corrected = y.copy() # Step 4: Correct the error in y by subtracting the scalar multiple
                y_corrected[i] = (y_corrected[i] - scalar) % radix
                return y_corrected.tolist()
    print("Unable to correct error.")
    return y.tolist()

def filter_output_by_indices(output, positions_array):
    filtered_values = [output[i-1] for i in positions_array]
    print(f'message: {filtered_values} specified by INFO BITS positions!!')
    
    
#v v v v v v v v v v v v v v v CHANGE THIS v v v v v v v v v v v v v v v v v v v v v
H = np.array([  # Parity check matrix (rows)
    [0,0,0,0,0,1],
    [1, 0,0,0,0,2],
    [0,1,1,1,0,0],
    [0,2,1,0,1,2]
])
y = [0,2,1,2,0,0] 
radix = 3
info_bits_positions = [1,2,4,6]

corrected_codeword = correct_error(H, y, radix)
print(f"Corrected codeword: {corrected_codeword}")
filter_output_by_indices(corrected_codeword, info_bits_positions)

Corrected codeword: [0, 2, 2, 2, 0, 0]
message: [0, 2, 2, 0], specified by INFO BITS POSITIONS


# q6

In [61]:
'''
let C be binary linear code with generator matrix G = ...
what is the minimal weight w(C) OR min distance d(C)? 
'''
import numpy as np
from itertools import combinations

def weight_of_codeword(codeword):
    return np.count_nonzero(codeword)

def generate_combinations(G, modulus):
    num_rows = G.shape[0]
    min_weight = float('inf')  # Initialize with a large number

    for r in range(1, num_rows + 1):  # r = 1 to num_rows
        for row_indices in combinations(range(num_rows), r):
            codeword = np.sum(G[list(row_indices)], axis=0) % modulus # Sum the selected rows modulo the modulus
            weight = weight_of_codeword(codeword) # Calculate the weight of the current codeword
            min_weight = min(min_weight, weight) # Update minimum weight
    return min_weight


#v v v v v v v v v v v v v v v CHANGE THIS v v v v v v v v v v v v v v v v v v v v v
G = np.array([
    [0,1,0,1,0],  # Row 1
    [1, 1, 1, 0,0],  # Row 2
    [0, 0, 1, 0, 1],  # Row 3
    [0, 1,0,0,1]   # Row 4
])
radix = 2

print(f"w(C) or d(C): {generate_combinations(G, radix)}")

w(C) or d(C): 1


# q6.2

In [71]:
'''
let C be the binary linear code with parity check matrix H =...
which of the following is a generator matrix for C?
'''
import numpy as np
def is_valid_generator_matrix(H, G, modulus):
    product = np.dot(H, G.T) % modulus # Step 1: Check if H * G^T (mod modulus) equals the zero matrix
    if np.count_nonzero(product) != 0:  # If any non-zero entries exist, it's invalid
        return False
    if np.any(np.all(G == 0, axis=1)): # Check if any row in G is all zeros
        return False
    if np.linalg.matrix_rank(G) < G.shape[0]: # Check if rows of G are linearly independent
        return False
    return True # If both conditions are satisfied, the matrix G is a valid generator matrix for C


#v v v v v v v v v v v v v v v CHANGE THIS v v v v v v v v v v v v v v v v v v v v v
H = np.array([
    [0, 0, 0, 0, 1],
    [1, 1, 1, 0, 2],
    [2, 1, 0, 1, 2]
])
G = np.array([ # test each G option
    [2,1,0,1,0],
    [0,1,2,2,0]
])
radix = 3

print(f"{'Yes' if is_valid_generator_matrix(H, G, radix) else 'No'}")

Yes


# q7

In [70]:
'''
let C be the binary linear code with basis B= {...} with information bits in positions 1,5,6. 
state the codeword x that encodes the message m= 101
'''
import numpy as np
def encode_message(m, B, info_bits_positions, radix):
    m = np.array(m)
    info_bits_positions = [pos - 1 for pos in info_bits_positions] # Map the 1-based positions to 0-based indices
    for basis_vector in B: # Check if any of the basis vectors already satisfy the message at the info bit positions
        if np.all(basis_vector[info_bits_positions] == m):
            return basis_vector

    n = len(B)
    for i in range(1, radix**n):
        coeffs = np.array([int(x) for x in np.base_repr(i, base=radix).zfill(n)], dtype=int)
        codeword = np.zeros(len(B[0]), dtype=int) # Linear combination of the basis vectors
        for j in range(n):
            codeword = (codeword + coeffs[j] * B[j]) % radix
        if np.all(codeword[info_bits_positions] == m): # Check if this codeword matches message at info bit positions
            return codeword
    return None


#v v v v v v v v v v v v v v v CHANGE THIS v v v v v v v v v v v v v v v v v v v v v
B = np.array([
    [0, 1,0,0,2],
    [2, 1, 0, 0, 1],
    [2,2,2, 1, 1]
])
m = [2,2,2]
info_bits_positions = [1, 4,5]
radix = 3

print(f"Codeword x: {encode_message(m, B, info_bits_positions, radix)}")

Codeword encoding the message: [2 2 1 2 2]


# q7.2

In [73]:
'''
consider the ternary linear code C with basis {...}. what is the minimum distance d(C) of C?
'''
import numpy as np
from itertools import product

def minimum_distance(B, radix=3):
    num_vectors, vector_length = B.shape # Number of basis vectors and their length
    min_weight = float('inf') # Store the minimum weight found (initialize with a large value)
    
    for coeffs in product(range(radix), repeat=num_vectors):
        if all(c == 0 for c in coeffs):# Skip the all-zero combination (this corresponds to the zero codeword)
            continue
        codeword = np.zeros(vector_length, dtype=int) # Compute the linear combination modulo radix
        for i, coeff in enumerate(coeffs):
            codeword = (codeword + coeff * B[i]) % radix

        weight = np.count_nonzero(codeword) # Calculate the weight of the codeword (number of non-zero entries)
        if weight < min_weight: # Update the minimum weight if the current weight is smaller
            min_weight = weight
    return min_weight


#v v v v v v v v v v v v v v v CHANGE THIS v v v v v v v v v v v v v v v v v v v v v
B = np.array([
    [0, 2, 1, 2, 1],
    [1, 1, 0, 0, 0],
    [2, 1, 1, 0, 0]
])
radix = 3

print(f"d(C): {minimum_distance(B, radix)}")

d(C): 2
